In [ ]:
pip install networkx

## Bibliotecas

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import json
import random
import pandas as pd

## Funções das métricas de centralidade

In [ ]:
def calculate_degree_centrality(G):
    centrality = nx.degree_centrality(G)
    return centrality

def calculate_closeness_centrality(G):
    centrality = nx.closeness_centrality(G)
    return centrality

def calculate_katz_centrality(G):
    return nx.katz_centrality(G)

def calculate_eigenvector_centrality(G):
    return nx.eigenvector_centrality(G)

def calculate_betweenness_centrality(G):
    return nx.betweenness_centrality(G)


## Leitura dos dados

In [ ]:
data = json.load(open('./dict_relacoes.json'))
amostras = [50, 100, 500]
metrics = [calculate_degree_centrality, calculate_closeness_centrality,
           calculate_katz_centrality, calculate_eigenvector_centrality,
           calculate_betweenness_centrality]

## Execução

In [ ]:
for k in amostras:
    G = nx.Graph()
    edges_final = list()
    for chave in data.keys():
        edges = data.get(chave, [])
        edges_final.extend(edges)
    random_edges = random.sample(edges_final, k=k)
    for edge in random_edges:
        if edge[0] != edge[1]:
            G.add_edge(*edge)
    dados_teste = {"k": k, "nodes": G.number_of_nodes(), "edges": G.number_of_edges()}
    pd.DataFrame(data=dados_teste.values(), index=dados_teste.keys(), columns=["dados"]).to_csv(f"dados_teste_{k}.csv")
    for metrica in metrics:
        try:
            results = metrica(G)
        except:
            pass
        else:
            metrica = str(metrica).split(" ")[1]
            pd.DataFrame(data=results.values(), index=results.keys(), columns=[metrica]).to_csv(f"{metrica}_{k}.csv")
            max_node = max(results, key=results.get)
            neighbors = list(G.neighbors(max_node))

            # Criar um subgrafo contendo apenas o vértice de maior grau e seus vizinhos
            subgraph_nodes = [max_node] + neighbors
            subgraph = G.subgraph(subgraph_nodes)
            # Plotar o subgrafo
            plt.figure(figsize=(8,8))

            # Posicionamento dos nós do subgrafo
            pos = nx.spring_layout(subgraph)

            # Desenhar as arestas do subgrafo
            nx.draw_networkx_edges(subgraph, pos, alpha=0.3)

            # O tamanho dos nós será proporcional à centralidade de grau no subgrafo
            node_size = [results[node] * 1000 for node in subgraph_nodes]
            node_color = ['orange' if node == max_node else 'skyblue' for node in subgraph]

            # Desenhar os nós do subgrafo
            nx.draw_networkx_nodes(subgraph, pos, node_size=node_size, node_color=node_color, alpha=0.9)

            # Criar os rótulos com os valores de centralidade
            labels = {node: f"{node}\n\n{results[node]:.3f}" for node in subgraph_nodes}

            # Desenhar os rótulos dos nós, incluindo os valores de centralidade
            nx.draw_networkx_labels(subgraph, pos, labels=labels, font_size=10)

            # Exibir o gráfico
            plt.title(f"Vértice de Maior Centralidade ({max_node}) e seus Vizinhos")
            plt.savefig(f"{metrica}_{k}.png")